# Project 3 - Part 1 (Data Cleaning)
## Roman Sielewicz, Imani Mufti, Alek Carlson, Adam Tassabehji, Sharath Reddy
### 2021 April 29

### Importing Packages

In [1]:
# Importing Packages
from datetime import datetime
start_time = datetime.now()
# import kaleido
import datetime as dt
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
# packages for google colab
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

### Loading Data from cloud file

In [3]:
# getting link and accessing data
# link: shareable link to excel file
link = 'https://drive.google.com/file/d/14bDUDVX4AKEtpFLwrNq5qv1Gcm9Ae3jJ/view?usp=sharing' 
id = link.split('/')[-2] # id: the drive file identifier
print(id)

14bDUDVX4AKEtpFLwrNq5qv1Gcm9Ae3jJ


In [4]:
# creating pandas dataframe
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('card transactions.xlsx')  
df = pd.read_excel('card transactions.xlsx')

In [ ]:
# # Alternative for running on local machine (uncomment to run)
# # Loading Data
# df = pd.read_excel('card transactions.xlsx')

In [5]:
# Information about the data
print(f'{df.shape[0]} \trows \n{df.shape[1]} \tcolumns')
# print(df.dtypes)
df.head()

96753 	rows 
10 	columns


,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0


### Initial Data Cleaning

In [6]:
# Renaming columns
df.rename(columns={'Merch description':'Merchdescription', 'Merch state':'Merchstate', 'Merch zip':'Merchzip'}, inplace=True)

In [7]:
# Remove this extremely large transaction amount--transaction value with over $3 million
df = df.loc[df['Amount'] <= 1000000].copy()
df.loc[df['Amount'] == df['Amount'].max(), :]

,Recnum,Cardnum,Date,Merchnum,Merchdescription,Merchstate,Merchzip,Transtype,Amount,Fraud
47339,47340,5142275225,2010-06-22,NaN,INTERSOL,NaN,NaN,P,47900.0,0


In [8]:
# Removing all transactioned where tranaction type is not 'P'
print(f'Before: {len(df)}')
df = df[df['Transtype'] == 'P']
print(f'After: {len(df)}')
df.head()

Before: 96752
After: 96397


,Recnum,Cardnum,Date,Merchnum,Merchdescription,Merchstate,Merchzip,Transtype,Amount,Fraud
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0


Imputing Missing Values

In [9]:
df.isna().sum()

Recnum                 0
Cardnum                0
Date                   0
Merchnum            3198
Merchdescription       0
Merchstate          1020
Merchzip            4300
Transtype              0
Amount                 0
Fraud                  0
dtype: int64

In [10]:
# replacing 0's with NaN
df['Merchnum'] = df['Merchnum'].replace({'0':np.nan})
# converting certain columns to string type
for c in ['Cardnum', 'Merchnum', 'Merchstate', 'Merchzip']:
    df[c] = df[c].astype(str)  

In [11]:
# defining most_frequent function 
def most_frequent(x):
    try:
        return x.value_counts().idxmax()
    except:
        return "nan"

In [ ]:
# Replacing null values for Merchnum using Merchdescription and Cardnum
colname = 'Merchnum'
for linkingname in ['Merchdescription','Cardnum']:
    initialnull = len(df[df[colname]=='nan'])
    df['mode_'+colname+'_groupby_'+linkingname] = df[linkingname].map(df.groupby(linkingname)[colname].agg(lambda x: most_frequent(x)))
    df[colname] = np.where(df[colname]=='nan', df['mode_'+colname+'_groupby_'+linkingname], df[colname])
    endnull = len(df[df[colname]=='nan'])
    print(f'{colname} - initial: {initialnull} null values, replaced: {initialnull-endnull} null values, remaining: {endnull} null values')

Merchnum - initial: 3251 null values, replaced: 361 null values, remaining: 2890 null values
Merchnum - initial: 2890 null values, replaced: 2114 null values, remaining: 776 null values


In [ ]:
# Replacing null values for Merchnum using Recnum
initialnull = len(df[df['Merchnum']=='nan'])
df['Merchnum'] = np.where(df['Merchnum']=='nan', df['Recnum'], df['Merchnum'])
endnull = df['Merchnum'].isnull().sum()
print(f'initial: {initialnull} null values, replaced: {initialnull-endnull} null values, remaining: {endnull} null values')

initial: 776 null values, replaced: 776 null values, remaining: 0 null values


In [ ]:
# Replacing null values for Merchstate and Merchzip using Merchnum
linkingname = 'Merchnum'
for colname in ['Merchstate', 'Merchzip']:
    initialnull = len(df[df[colname]=='nan'])
    df['mode_'+colname+'_groupby_'+linkingname] = df[linkingname].map(df.groupby(linkingname)[colname].agg(lambda x: most_frequent(x)))
    df[colname] = np.where(df[colname]=='nan', \
                           df['mode_'+colname+'_groupby_'+linkingname], df[colname])
    endnull = len(df[df[colname]=='nan'])
    print(f'{colname} - initial: {initialnull} null values, \
    replaced: {initialnull-endnull} null values, remaining: {endnull} null values')

Merchstate - initial: 1020 null values,     replaced: 603 null values, remaining: 417 null values
Merchzip - initial: 4300 null values,     replaced: 1592 null values, remaining: 2708 null values


In [ ]:
# Replacing null values for Merchstate and Merchzip using Recnum
for colname in ['Merchstate', 'Merchzip']: 
    initialnull = len(df[df[colname]=='nan'])
    df[colname] = np.where(df[colname]=='nan', \
                             df['Recnum'], df[colname])
    endnull = len(df[df[colname]=='nan'])
    print(f'{colname} - initial: {initialnull} null values, replaced: \
    {initialnull-endnull} null values, remaining: {endnull} null values')

Merchstate - initial: 417 null values, replaced:     417 null values, remaining: 0 null values
Merchzip - initial: 2708 null values, replaced:     2708 null values, remaining: 0 null values


In [ ]:
df.drop(columns=['mode_Merchnum_groupby_Merchdescription', \
                   'mode_Merchnum_groupby_Cardnum', \
                   'mode_Merchstate_groupby_Merchnum', \
                   'mode_Merchzip_groupby_Merchnum'], \
          inplace=True)

In [ ]:
df.head()

,Recnum,Cardnum,Date,Merchnum,Merchdescription,Merchstate,Merchzip,Transtype,Amount,Fraud
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0


In [ ]:
df.isna().sum()

Recnum              0
Cardnum             0
Date                0
Merchnum            0
Merchdescription    0
Merchstate          0
Merchzip            0
Transtype           0
Amount              0
Fraud               0
dtype: int64

In [ ]:
df.to_csv('card_transactions_clean.csv', index = False)

In [ ]:
# to export csv to google drive
folder_link = 'https://drive.google.com/drive/folders/1iraC2IH38ybBVFi1BQQbN6rA2_rVmE7-'
id = folder_link.split('/')[-1] # id: the drive file identifier
print(id)

1iraC2IH38ybBVFi1BQQbN6rA2_rVmE7-


In [ ]:
# Uploading to Google Drive
upload_file = 'card_transactions_clean.csv'
gfile = drive.CreateFile({'parents': [{'id': id}]})
# Read file and set it as the content of this instance.
gfile.SetContentFile(upload_file)
gfile.Upload() # Upload the file.

In [ ]:
print('Time elapsed: ', dt.datetime.now() - start_time)

Time elapsed:  0:00:49.304986
